In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

from google.colab import files
files.upload()

In [3]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d vipoooool/new-plant-diseases-dataset

100% 2.70G/2.70G [02:57<00:00, 17.1MB/s]
100% 2.70G/2.70G [02:57<00:00, 16.3MB/s]


In [5]:
%%capture
!unzip new-plant-diseases-dataset.zip

In [6]:
#import of the libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob
import cv2

In [7]:
plants = ['Apple', 'Maize', 'Potato']

In [8]:
diseases = []

In [9]:
trn_path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
for i in os.listdir(trn_path):
  if i.split('__')[0]=='Potato':
    diseases.append(i)

In [10]:
diseases

['Potato___healthy', 'Potato___Late_blight', 'Potato___Early_blight']

In [11]:
path = []
labels  = []
for dis in diseases:
  for i in os.listdir(os.path.join(trn_path, dis)):
    path.append(trn_path+'/'+dis+'/'+i)
    labels.append(dis)
df = pd.DataFrame(zip(path, labels)) 

In [12]:
df.columns = ['path', 'label']

In [13]:
# plt.imshow(cv2.imread('/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Black_rot/34eb19d8-32a9-4c9b-829f-d0ed705c5f03___JR_FrgE.S 3064_90deg.JPG'))

In [14]:
valid_path = '/content/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'

In [15]:
path = []
labels  = []
for dis in diseases:
  for i in os.listdir(os.path.join(valid_path, dis)):
    path.append(valid_path+'/'+dis+'/'+i)
    labels.append(dis)
df_test = pd.DataFrame(zip(path, labels)) 

In [16]:
df_test.columns = ['path', 'label']

In [17]:
df_test.head()

,path,label
0,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
1,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
2,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
3,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
4,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy


In [18]:
df.head()

,path,label
0,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
1,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
2,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
3,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy
4,/content/New Plant Diseases Dataset(Augmented)...,Potato___healthy


In [19]:
df.label.value_counts()

Potato___Late_blight     1939
Potato___Early_blight    1939
Potato___healthy         1824
Name: label, dtype: int64

In [20]:
#import pytorch libraries
import torchvision
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as T
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
from torchvision.models import vgg16
from sklearn.model_selection import train_test_split
import torch.nn.functional as Fun

In [21]:
lb = LabelEncoder()
df_test.label =  lb.fit_transform(df_test.label)
df.label =  lb.fit_transform(df.label)
dict(zip(lb.classes_, lb.transform(lb.classes_)))

{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [22]:
potato_classes = dict(zip(lb.classes_, lb.transform(lb.classes_)))
potato_classes

{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}

In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [27]:
class PotatoDataset(Dataset):
    def __init__(self, df):
        self.fpath = df.path
        self.label = df.label
        self.transforms = T.Compose(
        [T.ToTensor(),
        T.Normalize(mean=[0.485,
           0.456, 0.406],std=[0.229, 0.224, 0.225])]
        )
    def __len__(self):
        return len(self.fpath)
    
    def __getitem__(self,ix):
        f = self.fpath[ix]
        label = self.label[ix]
        im = cv2.imread(f)
        im = cv2.resize(im, (224,224))
        # im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        im = self.transforms(im)
        
        return torch.tensor(im,dtype=torch.float).to(device), torch.tensor(label, dtype=torch.float).long().to(device)
        

In [28]:
def get_model():
    model = torchvision.models.vgg16(pretrained=True)
    for param in model.features.parameters():
                   param.requires_grad = False
    model.avgpool = model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(nn.Flatten(),
                                   nn.Linear(512, 128),
                                   nn.ReLU(),
                                   nn.Dropout(0.2),
                                   nn.Linear(128, 3),
                                   nn.Softmax())
    model = model.to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [29]:
model, loss_fn, optimizer = get_model()
from torchsummary import summary

In [30]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [31]:
def get_data():
    train = PotatoDataset(df)
    test = PotatoDataset(df_test)
    trn_dl = DataLoader(train, batch_size=16,shuffle=True, drop_last=True)
    test_dl = DataLoader(test, batch_size=16, shuffle=True, drop_last=True)
    return trn_dl, test_dl

In [32]:
def train_batch(x, y, model, loss_fn, opt):
    model.train()
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()

In [33]:
@torch.no_grad()
def accuracy(x, y, model):
    model.eval()
    pred = model(x)
    _, is_correct = torch.max(pred.data,1)
    acc_all = (is_correct==y).detach().cpu().numpy()
    return acc_all

In [34]:
@torch.no_grad()
def val_loss_trn(x, y, model, loss_fn):
  prediction = model(x)
  val_loss = loss_fn(prediction, y)
  return val_loss.item()

In [35]:
trn_ldr, test_ldr = get_data()


In [36]:
train_loss = []
train_accuracies= []
val_loss = [] 
val_accuracies = []

for i in range(20):
    print(f'Epoch: _________*****{i}*****_______')
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies, val_epoch_losses = [], []



    for ix, batch in (enumerate(iter(trn_ldr))):
        
        x, y = batch
        # x = x.permute(0, 3,1,2)
        train_epoch_losses.append(train_batch(x, y, model, loss_fn, optimizer))
    train_epoch_loss = np.array(train_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Training Loss : {train_epoch_loss} *****_______')



    for ix, batch in (enumerate(iter(trn_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        train_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    train_epoch_accuracy = np.array(train_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Training Accuracy: {train_epoch_accuracy} *****_______')


    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_losses.append(val_loss_trn(x, y, model, loss_fn))
    val_epoch_loss = np.array(val_epoch_losses).mean()
    print(f'Epoch: _________*****{i} Validation Loss : {val_epoch_loss} *****_______')

    for ix, batch in (enumerate(iter(test_ldr))):
        x, y = batch
        # x = x.permute(0, 3,1,2)
        val_epoch_accuracies.append(sum(accuracy(x, y, model)) / len(y))
    val_epoch_accuracy = np.array(val_epoch_accuracies).mean()
    print(f'Epoch: _________*****{i} Validation Accuracy. {val_epoch_accuracy} *****_______')
    print('\n')

    train_loss.append(train_epoch_loss)
    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)
    val_loss.append(val_epoch_loss)

Epoch: _________*****0*****_______


<ipython-input-27-618a948cf1e5>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(im,dtype=torch.float).to(device), torch.tensor(label, dtype=torch.float).long().to(device)
/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:204: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch: _________*****0 Training Loss : 0.669400406854876 *****_______
Epoch: _________*****0 Training Accuracy: 0.9768258426966292 *****_______
Epoch: _________*****0 Validation Loss : 0.5891141349010254 *****_______
Epoch: _________*****0 Validation Accuracy. 0.9747191011235955 *****_______


Epoch: _________*****1*****_______
Epoch: _________*****1 Training Loss : 0.5852822957748778 *****_______
Epoch: _________*****1 Training Accuracy: 0.9840238764044944 *****_______
Epoch: _________*****1 Validation Loss : 0.5743270878041729 *****_______
Epoch: _________*****1 Validation Accuracy. 0.985252808988764 *****_______


Epoch: _________*****2*****_______
Epoch: _________*****2 Training Loss : 0.5760373118218411 *****_______
Epoch: _________*****2 Training Accuracy: 0.9833216292134831 *****_______
Epoch: _________*****2 Validation Loss : 0.5721860367260622 *****_______
Epoch: _________*****2 Validation Accuracy. 0.9831460674157303 *****_______


Epoch: _________*****3*****_______
Epoch: __

In [37]:
torch.save(model.to('cpu').state_dict(), '/content/potato.pth')

In [39]:
transforms = T.Compose(
        [T.ToTensor(),
        T.Normalize(mean=[0.485,
           0.456, 0.406],std=[0.229, 0.224, 0.225])]
        )

    
def read_transform(img):
        f = img
        im = cv2.imread(f)
        im = cv2.resize(im, (224,224))
        im = transforms(im)
        return torch.tensor(im,dtype=torch.float).to(device)

def get_model(no_classes):
    model = torchvision.models.vgg16(pretrained=True)
    for param in model.features.parameters():
                   param.requires_grad = False
    model.avgpool = model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(nn.Flatten(),
                                   nn.Linear(512, 128),
                                   nn.ReLU(),
                                   nn.Dropout(0.2),
                                   nn.Linear(128, no_classes),
                                   nn.Sigmoid())

    return model.to(device)

def load_model(model_path, classes):
  model = get_model(classes)
  state_dict = torch.load(model_path)
  return model,state_dict

In [40]:
def predict(image, crop):
  img = read_transform(image)
  img = img.to(device)
  model, state_dict = load_model('/content/potato.pth', 3)
  model.load_state_dict(state_dict)
  output = model(img.unsqueeze_(0))
  pred, conf = output.max(-1)
  
  return conf.item(), pred

In [41]:
predict('/content/potato early blight.jpeg', 'rice')

<ipython-input-39-8f7dfe928870>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(im,dtype=torch.float).to(device)
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


(0, tensor([1.], device='cuda:0', grad_fn=<MaxBackward0>))

In [42]:
predict('/content/potato-late-blight.webp', 'rice')

<ipython-input-39-8f7dfe928870>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(im,dtype=torch.float).to(device)


(1, tensor([0.9999], device='cuda:0', grad_fn=<MaxBackward0>))

In [45]:
predict('/content/potato healthy.webp', 'rice')

<ipython-input-39-8f7dfe928870>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(im,dtype=torch.float).to(device)


(2, tensor([1.0000], device='cuda:0', grad_fn=<MaxBackward0>))